##### Maybe we'll need to collect other few weather columns like visibility (https://github.com/ekapope/WorldWeatherOnline)

# Weather data using meteo stat package

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sys.path.append('../scripts')
from feature_transform import scrap_weather
from datetime import datetime
from meteostat import Stations, Daily, Point

# collecting weather data for each airport within X_train time range

In [2]:
flights = pd.read_csv('../data/flights_train.csv',
                      parse_dates=['flight_date'
                      ]).reset_index(drop=True)
url = \
    'https://raw.githubusercontent.com/ravisurdhar/flight_delays/master/airports.csv'
us_airports = pd.read_csv(url)


In [155]:
airports = np.unique(np.append(flights['from'].unique(), flights['to'
                     ].unique()))
airport_localisation = us_airports[us_airports['IATA_CODE'
                                   ].isin(airports)]


In [150]:
lats = airport_localisation['LATITUDE'].tolist()
longs = airport_localisation['LONGITUDE'].tolist()
iatas = airports.tolist()
list_geo = list(zip(lats, longs))
iatas = airports.tolist()
list_data = []

for i in range(len(list_geo)):
    point = Point(list_geo[i][0], list_geo[i][1])
    start = datetime(2011, 9, 1)
    end = datetime(2012, 11, 14)
    data = Daily(point, start, end)
    df = data.fetch()
    df = df.drop(labels=['snow', 'wdir', 'wpgt', 'pres', 'tsun'],
                 axis=1)
    list_data.append(df.reset_index())

weather_data = pd.concat(list_data, axis=0)


In [151]:
iatas_repeated = [iata for iata in iatas for i in
                  range(list_data[0].shape[0])]
weather_data['airport_code'] = iatas_repeated


In [147]:
weather_data.to_csv('../data/weather_data.csv')

In [171]:
weather_data

,time,tavg,tmin,tmax,prcp,wspd,airport_code
0,2011-09-01,27.8,21.7,33.9,0.0,4.0,ATL
1,2011-09-02,27.1,19.4,35.0,0.0,3.5,ATL
2,2011-09-03,27.7,20.6,35.0,0.0,5.6,ATL
3,2011-09-04,23.7,20.6,26.7,6.1,4.4,ATL
4,2011-09-05,23.7,22.2,25.6,16.0,10.8,ATL
...,...,...,...,...,...,...,...
436,2012-11-10,11.6,8.9,15.0,0.0,16.9,SFO
437,2012-11-11,10.6,6.1,15.6,0.0,3.6,SFO
438,2012-11-12,11.5,6.1,17.2,0.0,5.0,SFO
439,2012-11-13,13.5,8.3,18.3,0.0,4.3,SFO
